# Task 1

In [ ]:
!pip -q install cirq

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cirq

* Using Cirq for Quantum Operations
    1) With 5 qubits 
    2) Apply Hadamard operation on every qubit 
    3) Apply CNOT operation on (0, 1), (1,2), (2,3), (3,4) 
    4) SWAP (0, 4) 
    5) Rotate X with pi/2 on any qubit 
    6) Plot the circuit 


In [3]:
# Making 5 named Qubits
import cirq
a = cirq.NamedQubit('1')
b = cirq.NamedQubit('2')
c = cirq.NamedQubit('3')
d = cirq.NamedQubit('4')
e = cirq.NamedQubit('5')

# Performing the necessary operations
operations = [cirq.H(a), cirq.H(b),cirq.H(c),cirq.H(d),cirq.H(e), cirq.CNOT(a, b),cirq.CNOT(b, c),cirq.CNOT(c, d),cirq.CNOT(d,e),cirq.SWAP(a,e),cirq.Rx(rads = 0.5 * np.pi)(a)]

#Plotting the Cirquit
print(cirq.Circuit(operations))


1: ───H───@───────────────×───Rx(0.5π)───
          │               │
2: ───H───X───@───────────┼──────────────
              │           │
3: ───H───────X───@───────┼──────────────
                  │       │
4: ───H───────────X───@───┼──────────────
                      │   │
5: ───H───────────────X───×──────────────


* Implement a second circuit with a framework of your choice:

1.   Apply a Hadmard gate to the first qubit
2.   rotate the second qubit by pi/3 around X
3.   Apply Hadamard gate to the third and fourth qubit
4.   Perform a swap test between the states of the first and second qubit |q1 q2> and the third and fourth qubit |q3 q4> 



    


In [6]:
import cirq
import numpy as np

# Create a quantum circuit with 4 qubits
qc = cirq.Circuit()
q = cirq.LineQubit.range(4)

# Apply Hadamard gate to the first qubit
qc.append(cirq.H(q[0]))

# Rotate the second qubit by pi/3 around X
qc.append(cirq.rx(np.pi/3).on(q[1]))

# Apply Hadamard gate to the third and fourth qubit
qc.append(cirq.H(q[2]))
qc.append(cirq.H(q[3]))

# Perform a swap test between the states of the first and second qubit |q1 q2> 
# and the third and fourth qubit |q3 q4>

# Initialize an ancilla qubit
anc = cirq.NamedQubit('anc')
qc.append(cirq.H(anc))

# Apply controlled swap gates
qc.append(cirq.CSWAP(q[0], q[1], anc))
qc.append(cirq.CSWAP(q[2], q[3], anc))

# Apply controlled-Hadamard gates
qc.append(cirq.H(q[0]).controlled_by(anc))
qc.append(cirq.H(q[1]).controlled_by(anc))
qc.append(cirq.H(q[2]).controlled_by(anc))
qc.append(cirq.H(q[3]).controlled_by(anc))

# Measure the ancilla qubit
qc.append(cirq.measure(anc, key='result'))

# Simulate the circuit and print the results
simulator = cirq.Simulator()
result = simulator.run(qc, repetitions=1000)
print(result.histogram(key='result'))

Counter({0: 529, 1: 471})


# Task 2

In [ ]:
!pip install -q energyflow
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.5/700.5 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 KB 24.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
import energyflow as ef
import numpy as np

# Load dataset
num_data = 1000
X, y = ef.qg_jets.load(num_data=num_data, pad=True, ncol=4)

# Preprocess the dataset
def create_graph(jet):
    # Here we create a simple graph using only the pt, rapidity, and azimuthal angle columns
    positions = jet[:, 1:3]
    features = torch.tensor(jet[:, :3], dtype=torch.float)

    # Calculate distances between all pairs of nodes (particles) and threshold to create edges
    dist_threshold = 0.3
    distance_matrix = np.linalg.norm(positions[:, np.newaxis] - positions, axis=2)
    edges = np.argwhere(distance_matrix < dist_threshold)

    # Remove self-loops
    edges = edges[edges[:, 0] != edges[:, 1]]

    undirected_edges = to_undirected(torch.tensor(edges, dtype=torch.long).t().contiguous())
    return Data(x=features, edge_index=undirected_edges)


graph_dataset = [create_graph(jet) for jet in X]
train_indices, test_indices = train_test_split(range(len(graph_dataset)), test_size=0.3, stratify=y, random_state=42)

# Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes, num_heads=4):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_features, hidden_channels, heads=num_heads)
        self.conv2 = GATConv(hidden_channels * num_heads, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Training function
def train(model, dataset, train_indices, epochs=25, batch_size=16, device='cpu'):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        train_loss = 0.0
        np.random.shuffle(train_indices)
        model.train()
        for i in range(0, len(train_indices), batch_size):
            batch_indices = train_indices[i: i + batch_size]
            batch = [dataset[idx] for idx in batch_indices]
            batch_labels = torch.tensor([y[idx] for idx in batch_indices], dtype=torch.long, device=device)
            
            optimizer.zero_grad()
            batch_logits = torch.stack([model(graph.to(device)).mean(dim=0) for graph in batch], dim=0)
            loss = loss_fn(batch_logits, batch_labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_indices) / batch_size
        print(f"Epoch: {epoch + 1}, Train Loss: {train_loss:.4f}")

#Calculating Accuracy
def accuracy(y_true, y_pred):
    assert len(y_true) == len(y_pred), "The lengths of true and predicted labels must match."
    correct = np.sum(y_true == y_pred)
    return correct / len(y_true)


def evaluate(model, dataset, test_indices, device='cpu'):
    model.eval()
    logits, labels = [], []
    with torch.no_grad():
        for idx in test_indices:
            graph = dataset[idx].to(device)
            label = y[idx]
            logit = model(graph).mean(dim=0)
            logits.append(logit.cpu().numpy())
            labels.append(label)

    logits = np.stack(logits)
    labels = np.array(labels)

    pred_probs = np.exp(logits)[:, 1]
    auc_roc = roc_auc_score(labels, pred_probs)
    pred_labels = np.argmax(logits, axis=1)
    cm = confusion_matrix(labels, pred_labels)
    acc = accuracy(labels, pred_labels)

    return auc_roc, cm, acc


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = 3
hidden_channels = 64
num_classes = 2

# Initialize and train GCN model
gcn_model = GCN(num_features, hidden_channels, num_classes).to(device)
print("Training the GCN model")
train(gcn_model, graph_dataset, train_indices, device=device)


# Initialize and train GAT model
gat_model = GAT(num_features, hidden_channels, num_classes).to(device)
print("Training GAT model")
train(gat_model, graph_dataset, train_indices, device=device)



# Evaluate GCN model
gcn_auc_roc, gcn_cm, gcn_acc = evaluate(gcn_model, graph_dataset, test_indices, device=device)
print("Performance Metrics for the GCN model")
print(f"GCN AUC-ROC: {gcn_auc_roc:.4f}")
print(f"GCN Confusion Matrix:\n{gcn_cm}")
print(f"GCN Accuracy: {gcn_acc:.4f}")

# Evaluate GAT model
gat_auc_roc, gat_cm, gat_acc = evaluate(gat_model, graph_dataset, test_indices, device=device)
print("Performance Metrics for the GAT model")
print(f"GAT AUC-ROC: {gat_auc_roc:.4f}")
print(f"GAT Confusion Matrix:\n{gat_cm}")
print(f"GAT Accuracy: {gat_acc:.4f}")

Training the GCN model
Epoch: 1, Train Loss: 0.6766
Epoch: 2, Train Loss: 0.6347
Epoch: 3, Train Loss: 0.6058
Epoch: 4, Train Loss: 0.5898
Epoch: 5, Train Loss: 0.5636
Epoch: 6, Train Loss: 0.5569
Epoch: 7, Train Loss: 0.5629
Epoch: 8, Train Loss: 0.5487
Epoch: 9, Train Loss: 0.5351
Epoch: 10, Train Loss: 0.5585
Epoch: 11, Train Loss: 0.5307
Epoch: 12, Train Loss: 0.5492
Epoch: 13, Train Loss: 0.5419
Epoch: 14, Train Loss: 0.5329
Epoch: 15, Train Loss: 0.5268
Epoch: 16, Train Loss: 0.5378
Epoch: 17, Train Loss: 0.5341
Epoch: 18, Train Loss: 0.5413
Epoch: 19, Train Loss: 0.5291
Epoch: 20, Train Loss: 0.5590
Epoch: 21, Train Loss: 0.5327
Epoch: 22, Train Loss: 0.5309
Epoch: 23, Train Loss: 0.5260
Epoch: 24, Train Loss: 0.5380
Epoch: 25, Train Loss: 0.5333
Training GAT model
Epoch: 1, Train Loss: 0.7333
Epoch: 2, Train Loss: 0.5595
Epoch: 3, Train Loss: 0.6021
Epoch: 4, Train Loss: 0.5585
Epoch: 5, Train Loss: 0.5359
Epoch: 6, Train Loss: 0.5613
Epoch: 7, Train Loss: 0.5577
Epoch: 8, Trai

The GCN model has an AUC-ROC score of 0.8883 and an accuracy of 0.7833, while the GAT model has an AUC-ROC score of 0.8799 and an accuracy of 0.7767.

The confusion matrix shows that the GCN model correctly predicted 101 + 134 = 235 events, while the GAT model correctly predicted 94 + 139 = 233 events. Both models misclassified 46 and 53 events, respectively.

It should be noted that the performance of the models can be improved by using hyperparameter tuning, adjusting the number of layers and the size of the hidden layer.

# Task 3

Quantum computing has always piqued my curiosity since I first encountered Shor's algorithm, a revolutionary method for factoring large numbers exponentially faster than classical algorithms. As a budding computer science engineer, I am eager to explore and contribute to this disruptive technology that holds immense potential.

My initial exposure to quantum computing involved using TensorFlow Quantum and Cirq libraries, which provided a solid foundation in understanding the intricacies of quantum systems. Currently, I am delving into the Pennylane library, which has opened up new avenues for me to explore the fascinating world of quantum machine learning.

I believe that quantum machine learning offers unprecedented opportunities to solve complex problems that were previously deemed unsolvable. One particular method I would love to work on is the development of hybrid quantum-classical algorithms, which can leverage the best of both worlds to deliver optimized solutions in areas such as optimization and pattern recognition.

As I continue to expand my knowledge in quantum computing, I am excited to uncover the untapped potential of this groundbreaking technology and make significant contributions to its advancement.

# Task 5

A Quantum Graph Neural Network(QGNN) is the quantum analogue of the classical graph neural networks. The are designed such as to use quantum phenomenon such as superposition to their advantage and process the graph structured data more efficiently.

An approach for creating QGNN for classifying quarks and gluons using the graph representation is as follows:

1) Embedding the input graph into a quantum state: We can use single-qubit gates such as RY ot RZ gates to encode the data.

2) Apply a parameterized quantum circuit: In this step we can capture the graph's structure through entangling gates such as CNOT gates. We choose the entangling gate based on the adjancency matrix of the graphical gata.

3) Measure the output quantum state: After applying the quantum operations, we can measure the output quantum state to obtain classical information. For example, you can use expectation values of observables to extract the output node or graph-level features.

4) Decode the output features and perform a classical post-processing step: This step involves using a classical neural network or other machine learning algorithms to make the final predictions for quark and gluon classification based on the output features from the QGNN circuit.

5) Train the QGNN circuit using a hybrid quantum-classical optimization approach: We can optimize the parameters of the quantum circuit using gradient-based optimization techniques, such as gradient descent or Adam. The gradients can be computed using classical backpropagation or using quantum parameter-shift rules.


Given below is simple implementation of a QGNN

In [10]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers

def quantum_graph_embedding(A, X, params, n_qubits):
    for i in range(n_qubits):
        qml.RY(np.pi * X[i], wires=i)
    
    for j in range(n_qubits):
        for k in range(j+1, n_qubits):
            if A[j][k] == 1:
                qml.CNOT(wires=[j, k])
                qml.RY(params[j][k], wires=k)
                qml.CNOT(wires=[j, k])

def qgnn_circuit(A, X, params, n_qubits, n_layers):
    wires = list(range(n_qubits))
    dev = qml.device("default.qubit", wires=wires)

    @qml.qnode(dev)
    def circuit(params):
        for layer in range(n_layers):
            quantum_graph_embedding(A, X, params[layer], n_qubits)
            RandomLayers(params[layer], wires=wires)

        return qml.probs(wires=wires)

    return circuit

In [15]:
# Create a sample adjacency matrix (A) and node feature matrix (X) for demonstration purposes
A = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
X = np.array([0.5, 0.3, 0.8])

# Number of qubits and layers in the QGNN circuit
n_qubits = len(A)
n_layers = 3

# Randomly initialize the parameters
params = np.random.uniform(0, 2 * np.pi, (n_layers, n_qubits, n_qubits))

# Get the circuit drawing
qgnn_circuit_instance = qgnn_circuit(A, X, params, n_qubits, n_layers)
qgnn_circuit_instance(params)
print(qml.draw(qgnn_circuit_instance)(params))

0: ──RY(1.57)─╭●───────────╭●─────────────────╭RandomLayers(M0)──RY(1.57)─╭●───────────╭●───
1: ──RY(0.94)─╰X──RY(4.49)─╰X─╭●───────────╭●─├RandomLayers(M0)──RY(0.94)─╰X──RY(4.97)─╰X─╭●
2: ──RY(2.51)─────────────────╰X──RY(4.06)─╰X─╰RandomLayers(M0)──RY(2.51)─────────────────╰X

───────────────╭RandomLayers(M1)──RY(1.57)─╭●───────────╭●─────────────────╭RandomLayers(M2)─┤ ╭Probs
────────────╭●─├RandomLayers(M1)──RY(0.94)─╰X──RY(5.47)─╰X─╭●───────────╭●─├RandomLayers(M2)─┤ ├Probs
───RY(0.45)─╰X─╰RandomLayers(M1)──RY(2.51)─────────────────╰X──RY(4.90)─╰X─╰RandomLayers(M2)─┤ ╰Probs


### Some corner cases that may occur in the data

#### Empty Graph
An event with only one particle detected

In [16]:
A = np.array([[0]])
X = np.array([0.0])
n_qubits = len(A)
n_layers = 3

params = np.random.uniform(0, 2 * np.pi, (n_layers, n_qubits, n_qubits))

qgnn_circuit_instance = qgnn_circuit(A, X, params, n_qubits, n_layers)
qgnn_circuit_instance(params)
print(qml.draw(qgnn_circuit_instance)(params))

0: ──RY(0.00)──RandomLayers(M0)──RY(0.00)──RandomLayers(M1)──RY(0.00)──RandomLayers(M2)─┤  Probs


#### Linear Graph
A situation where particles interact sequentially, with each particle primarily interacting with its neighbors in the chain. 

In [17]:
A = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
X = np.array([0.1, 0.2, 0.3])

n_qubits = len(A)
n_layers = 3

params = np.random.uniform(0, 2 * np.pi, (n_layers, n_qubits, n_qubits))

qgnn_circuit_instance = qgnn_circuit(A, X, params, n_qubits, n_layers)
qgnn_circuit_instance(params)
print(qml.draw(qgnn_circuit_instance)(params))

0: ──RY(0.31)─╭●───────────╭●─────────────────╭RandomLayers(M0)──RY(0.31)─╭●───────────╭●───
1: ──RY(0.63)─╰X──RY(4.86)─╰X─╭●───────────╭●─├RandomLayers(M0)──RY(0.63)─╰X──RY(2.26)─╰X─╭●
2: ──RY(0.94)─────────────────╰X──RY(3.88)─╰X─╰RandomLayers(M0)──RY(0.94)─────────────────╰X

───────────────╭RandomLayers(M1)──RY(0.31)─╭●───────────╭●─────────────────╭RandomLayers(M2)─┤ ╭Probs
────────────╭●─├RandomLayers(M1)──RY(0.63)─╰X──RY(2.29)─╰X─╭●───────────╭●─├RandomLayers(M2)─┤ ├Probs
───RY(4.19)─╰X─╰RandomLayers(M1)──RY(0.94)─────────────────╰X──RY(0.64)─╰X─╰RandomLayers(M2)─┤ ╰Probs


# Task 8 

In [14]:
import numpy as np

from tqdm import tqdm, trange

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST

np.random.seed(0)
torch.manual_seed(0)


def patchify(images, n_patches):
    n, c, h, w = images.shape

    assert h == w, "Patchify method is implemented for square images only"

    patches = torch.zeros(n, n_patches ** 2, h * w * c // n_patches ** 2)
    patch_size = h // n_patches

    for idx, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:, i * patch_size: (i + 1) * patch_size, j * patch_size: (j + 1) * patch_size]
                patches[idx, i * n_patches + j] = patch.flatten()
    return patches


class MyMSA(nn.Module):
    def __init__(self, d, n_heads=2):
        super(MyMSA, self).__init__()
        self.d = d
        self.n_heads = n_heads

        assert d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"

        d_head = int(d / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)

                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])


class MyViTBlock(nn.Module):
    def __init__(self, hidden_d, n_heads, mlp_ratio=4):
        super(MyViTBlock, self).__init__()
        self.hidden_d = hidden_d
        self.n_heads = n_heads

        self.norm1 = nn.LayerNorm(hidden_d)
        self.mhsa = MyMSA(hidden_d, n_heads)
        self.norm2 = nn.LayerNorm(hidden_d)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, mlp_ratio * hidden_d),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_d, hidden_d)
        )

    def forward(self, x):
        out = x + self.mhsa(self.norm1(x))
        out = out + self.mlp(self.norm2(out))
        return out


class MyViT(nn.Module):
    def __init__(self, chw, n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10):
        # Super constructor
        super(MyViT, self).__init__()
        
        # Attributes
        self.chw = chw # ( C , H , W )
        self.n_patches = n_patches
        self.n_blocks = n_blocks
        self.n_heads = n_heads
        self.hidden_d = hidden_d
        
        # Input and patches sizes
        assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)

        # 1) Linear mapper
        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)
        
        # 2) Learnable classification token
        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))
        
        # 3) Positional embedding
        self.register_buffer('positional_embeddings', get_positional_embeddings(n_patches ** 2 + 1, hidden_d), persistent=False)
        
        # 4) Transformer encoder blocks
        self.blocks = nn.ModuleList([MyViTBlock(hidden_d, n_heads) for _ in range(n_blocks)])
        
        # 5) Classification MLPk
        self.mlp = nn.Sequential(
            nn.Linear(self.hidden_d, out_d),
            nn.Softmax(dim=-1)
        )

    def forward(self, images):
        # Dividing images into patches
        n, c, h, w = images.shape
        patches = patchify(images, self.n_patches).to(self.positional_embeddings.device)
        
        # Running linear layer tokenization
        # Map the vector corresponding to each patch to the hidden size dimension
        tokens = self.linear_mapper(patches)
        
        # Adding classification token to the tokens
        tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)
        
        # Adding positional embedding
        out = tokens + self.positional_embeddings.repeat(n, 1, 1)
        
        # Transformer Blocks
        for block in self.blocks:
            out = block(out)
            
        # Getting the classification token only
        out = out[:, 0]
        
        return self.mlp(out) # Map to output dimension, output category distribution
    


def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result


def main():
    # Loading data
    transform = ToTensor()

    train_set = MNIST(root='./../datasets', train=True, download=True, transform=transform)
    test_set = MNIST(root='./../datasets', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
    test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

    # Defining model and training options
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")
    model = MyViT((1, 28, 28), n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10).to(device)
    N_EPOCHS = 5
    LR = 0.005

    # Training loop
    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()
    for epoch in trange(N_EPOCHS, desc="Training"):
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training",position=0,leave=True):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)

            train_loss += loss.detach().cpu().item() / len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")

    # Test loop
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0
        for batch in tqdm(test_loader, desc="Testing"):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            test_loss += loss.detach().cpu().item() / len(test_loader)

            correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
            total += len(x)
        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {correct / total * 100:.2f}%")


if __name__ == '__main__':
    main()

Using device:  cpu 


Training:  20%|███████                            | 1/5 [01:13<04:54, 73.58s/it]

Epoch 1/5 loss: 2.11


Training:  40%|██████████████                     | 2/5 [02:27<03:41, 73.82s/it]

Epoch 2/5 loss: 1.84


...

Epoch 4/5 loss: 1.72


Training: 100%|███████████████████████████████████| 5/5 [06:05<00:00, 73.03s/it]


Epoch 5/5 loss: 1.68


Testing: 100%|██████████████████████████████████| 79/79 [00:06<00:00, 11.50it/s]

Test loss: 1.73
Test accuracy: 72.58%


Extending a classical vision transformer to a quantum vision transformer would require incorporating quantum mechanics into the model's architecture. This would likely involve the use of quantum algorithms and quantum hardware, such as quantum circuits and qubits, to perform the computations.

Here is a high-level sketch of what a quantum vision transformer architecture might look like:

1) Quantum Image Encoding: The first step in the architecture would be to encode the classical image data into a quantum state. This could be done using quantum image encoding techniques, such as the quantum Fourier transform, to map the classical image pixels to quantum states.

2) Quantum Attention Mechanisms: The next step would be to perform quantum attention mechanisms to process the quantum image encoding. This could be done using quantum circuits that perform quantum transformations, such as the quantum phase estimation algorithm, to compute attention scores for the different regions of the image.

3) Quantum Layers: The quantum vision transformer would then consist of multiple quantum layers, each of which would perform quantum transformations on the quantum image encoding. These transformations could be based on quantum algorithms, such as the quantum alternating operator ansatz or the quantum circuit-centric approach, to process the quantum image encoding.

4) Quantum Pooling: After the quantum layers have processed the quantum image encoding, a quantum pooling operation would be performed to aggregate the information from the different regions of the image. This could be done using quantum algorithms, such as the quantum maximum finding algorithm, to compute the maximum value for each region of the image.

5) Quantum Classification: Finally, the quantum image encoding would be passed through a quantum classifier to make the final prediction. This could be done using quantum algorithms, such as the quantum support vector machine or quantum neural networks, to classify the image based on its quantum encoding.

It's worth noting that this is just one possible approach to designing a quantum vision transformer architecture, and there are many other ways to incorporate quantum mechanics into the model's architecture. Additionally, quantum hardware and software technologies are still in their early stages of development, so there may be technical limitations to implementing a full-fledged quantum vision transformer at this time. However, as quantum technologies continue to advance, it may become possible to build more sophisticated quantum vision transformers in the future.